<a href="https://colab.research.google.com/github/kmaciver/Ryerson_Capstone/blob/master/Approach/Step2-DayTrade/DayTradeSinglePrediction_ver3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler, StandardScaler, QuantileTransformer

In [0]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding, LSTM, TimeDistributed, Lambda
from tensorflow.python.keras.optimizers import RMSprop, Adam
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras import losses

Using TensorFlow backend.


In [0]:
file_path = "/content/drive/My Drive/Capstone/Data Exploration/Day_trade_data.csv"
DayTrade = pd.read_csv(file_path, index_col='Time')
DayTrade = DayTrade.drop([DayTrade.columns[0]] ,  axis='columns')
DayTrade.head()

,Open,High,Low,Close,Volume_.BTC.,Volume_.Currency.,Weighted_Price,date,Open_RoC,High_RoC,Low_RoC,Close_RoC,Weighted_Price_RoC,RSI,MACD_index,slow_stoch
Time,,,,,,,,,,,,,,,,
2013-04-03 00:00:00,116.00,116.00,116.00,116.00,31.713233,3678.735005,116.000000,2013-04-03,0.000000,0.000000,0.000000,0.000000,0.000000,33.333333,-0.36038,0.084906
2013-04-03 00:01:00,116.00,116.00,116.00,116.00,31.713233,3678.735005,116.000000,2013-04-03,0.000000,0.000000,0.000000,0.000000,0.000000,33.333333,-0.36038,0.084906
2013-04-03 00:02:00,116.00,116.58,116.00,116.58,2.050985,238.357034,116.215883,2013-04-03,0.000000,0.004988,0.000000,0.004988,0.001859,33.333333,-0.36038,0.084906
2013-04-03 00:03:00,116.98,117.00,116.98,117.00,23.000000,2690.890000,116.995217,2013-04-03,0.008413,0.003596,0.008413,0.003596,0.006684,33.333333,-0.36038,0.084906
2013-04-03 00:04:00,117.00,117.00,117.00,117.00,50.000000,5850.000000,117.000000,2013-04-03,0.000171,0.000000,0.000171,0.000000,0.000041,33.333333,-0.36038,0.084906


Dropping Volume Currency as discussed in the Feature Selection phase

In [0]:
DayTrade = DayTrade.drop(columns='Volume_.Currency.')

In [0]:
#We need create a target data, which is basically a copy of the data that will be later shifted
target_data = DayTrade.copy()
target_data = target_data.iloc[:,5:7]
target_data.head()

,Weighted_Price,date
Time,,
2013-04-03 00:00:00,116.000000,2013-04-03
2013-04-03 00:01:00,116.000000,2013-04-03
2013-04-03 00:02:00,116.215883,2013-04-03
2013-04-03 00:03:00,116.995217,2013-04-03
2013-04-03 00:04:00,117.000000,2013-04-03


The objective of the model is to predict 10 minutes ahead of the current timestep. The Day Trade data contains the minute to minute data for a total of 1735 days. The analysis must be limited within each day.

In [0]:
# Predict 10 minutes in the future, although the predictions must be wrapped around each day
shift_steps = 10

# Now that the target_data was created we need to shift the data so that the target values of 24 hours later aling with our
# input data

target_data = target_data.groupby('date').shift(-shift_steps)


Here we double check that because we shifted the target values now we have NaN values at the end

In [0]:
target_data.iloc[1420:1450]

,Weighted_Price
Time,
2013-04-03 23:40:00,129.900000
2013-04-03 23:41:00,129.900000
2013-04-03 23:42:00,129.899861
2013-04-03 23:43:00,129.892440
2013-04-03 23:44:00,130.049341
2013-04-03 23:45:00,131.371316
2013-04-03 23:46:00,132.534018
2013-04-03 23:47:00,132.912123
2013-04-03 23:48:00,132.819273


In [0]:
# Now we need to remove the rows with NaN values for the target data thus needing to exclude also the 
# 10 lines per day of the DayTrade data

DayTrade['target'] = target_data['Weighted_Price']

target_data['date'] = DayTrade['date']

DayTrade_clean = DayTrade.dropna()
target_data_clean = target_data.dropna()

In [0]:
DayTrade_clean.shape, target_data_clean.shape

((2481050, 16), (2481050, 2))

Number of rows for both data are correct since initially the data consisted of 1735 days of 1440 minutes (total of 2.481.050 rows) and now each day had the last 10 minutes so the total amount of rows must be 1735 days of 1430 minutes (total of 2.481.050)

In [0]:
# the target column was only added to the DayTrade data in order to drop the correct rows.
#Removing column 'target' from the DayTrade data

DayTrade = DayTrade.drop(columns='target')
DayTrade_clean = DayTrade_clean.drop(columns='target')

In order to compare results with other models a 90% split is going to be made

In [0]:
days_in_data = list(dict.fromkeys(DayTrade_clean["date"].values))

split = 0.9

training_days = days_in_data[:int(split*len(days_in_data))]
testing_days = days_in_data[int(split*len(days_in_data)):]

print(len(training_days),len(testing_days))

1561 174


Data will need to be normalized for predictions. A scaler will be fitted for the training data

In [0]:
#Step 1 - Convert day data into numpy array
train_data = np.array(DayTrade_clean.loc[DayTrade_clean.date.isin(training_days),:].drop(columns='date'))
label_data = np.array(target_data_clean.loc[target_data_clean.date.isin(training_days),:].drop(columns='date')).reshape(-1,1)

#Step 2 - Scale data for Neural Network
x_scaler = MinMaxScaler()
x_scaler.fit_transform(train_data)
y_scaler = MinMaxScaler()
y_scaler.fit_transform(label_data)

array([[0.00331603],
       [0.00331603],
       [0.00331603],
       ...,
       [0.19470373],
       [0.19471025],
       [0.19481153]])

In [0]:
def batch_reshape(sequence_length, X_train_scale, Y_train_scale, num_x_signal, num_y_signal):
    """
    Generator function for creating random batches of training-data.
    """
    batch_size = X_train_scale.shape[1] // sequence_length
    # Allocate a new array for the batch of input-signals.
    x_shape = (batch_size, sequence_length, num_x_signal)
    x_batch = np.zeros(shape=x_shape, dtype=np.float16)
    
 
    # Allocate a new array for the batch of output-signals.
    y_shape = (batch_size, num_y_signal)
    y_batch = np.zeros(shape=y_shape, dtype=np.float16)

    #print(x_batch.shape, y_batch.shape, X_train_scale.shape, Y_train_scale.shape) #debugging
    # Create Sequence for sliding window
    seq = []
    for i in range(batch_size):
        seq.append(i*sequence_length)
    
    # Fill the batch with sequences of data.
    for i in range(0,len(seq)-1):

        # Copy the sequences of data starting at this index.
        x_batch[i] = X_train_scale[0][seq[i]:seq[i]+sequence_length][:]
        y_batch[i] = Y_train_scale[0][seq[i]+sequence_length-1][:]
        #print("iteration: ",i,"-OK") #debugging

    #print(x_batch.shape,y_batch.shape) #debbuging
    return (x_batch, y_batch)

In [0]:
import random as rand
from random import randint
from random import seed
rand.seed(4)


def batch_generator(batch_size, sequence_length, num_x_signal, num_y_signal, train_data, label_data, training_days):
    
    # Create a Batch function for training data using sliding window technique
    # Step 1 - Select a training day

    day = rand.choice(training_days)

    #Step 2 - Filter Input data and Target with the selected date
    day_train_data = train_data[train_data['date'].values==day]
    day_label_data = label_data[label_data['date'].values==day]

    #Step 3 - Drop date columns from day train data and day label data
    day_train_data = day_train_data.drop(columns='date')
    day_label_data = day_label_data.drop(columns='date')

    #Step 4 - Convert day data into numpy array
    day_train_data = np.array(day_train_data)
    day_label_data = np.array(day_label_data).reshape(-1,1)

    #Step 5 - Scale data for Neural Network
    day_train_data = x_scaler.fit_transform(day_train_data)
    day_label_data = y_scaler.fit_transform(day_label_data)

    #Step 6 - Reshape data to fit keras requirement to have a (x,y,z) shape
    day_train_data = day_train_data.reshape(1,day_train_data.shape[0],day_train_data.shape[1])
    day_label_data = day_label_data.reshape(1,day_label_data.shape[0],day_label_data.shape[1])
    
     #Step 7 - Reshape data into Batches
    day_train_data_reshape , day_label_data_reshape =  batch_reshape(sequence_length, day_train_data, day_label_data, num_x_signal, num_y_signal)
    
    # print(day_train_data_reshape.shape , day_train_data_reshape.shape)#debugging
   
    #Step 8 - Apply the "jumping" slidding window technique to the reshaped date
    # Infinite loop.
    while True:
        # Allocate a new array for the batch of input-signals.
        x_shape = (batch_size, sequence_length, num_x_signal)
        x_batch = np.zeros(shape=x_shape, dtype=np.float16)

        # Allocate a new array for the batch of output-signals.
        y_shape = (batch_size, num_y_signal)
        y_batch = np.zeros(shape=y_shape, dtype=np.float16)

        # Fill the batch with random continuous sequences of data.

        # Get a random start-index.
        # This points somewhere into the training-data.
        idx = np.random.randint(day_train_data_reshape.shape[0] - batch_size)

        # Copy the sequences of data starting at this index.
        x_batch = day_train_data_reshape[idx:idx+batch_size]
        y_batch = day_label_data_reshape[idx:idx+batch_size]


        yield (x_batch, y_batch)



In [0]:
num_x_signal = 14 # number of input features
num_y_signal = 1 # number of label classes

batch_size = 50 # tunning parameter
sequence_length = 25 #Amount of time-steps to look back for the 10 minute prediction

In [0]:
generator = batch_generator(batch_size,sequence_length, num_x_signal, num_y_signal, DayTrade_clean, target_data_clean,training_days)

In [0]:
x_batch, y_batch = next(generator)

In [0]:
print(x_batch.shape)
print(y_batch.shape)

(50, 25, 14)
(50, 1)


In [0]:

def batch_validation(sequence_length, num_x_signal, num_y_signal, test_data, label_test_data, testing_days):
    
    # Create a Batch function for validation data using sliding window technique
    # Step 1 - Select a testing day

    day = rand.choice(testing_days)

    #Step 2 - Filter Input data and Target with the selected date
    day_test_data = test_data[test_data['date'].values==day]
    day_label_data = label_test_data[label_test_data['date'].values==day]

    #Step 3 - Drop date columns from day test data and day label data
    day_test_data = day_test_data.drop(columns='date')
    day_label_data = day_label_data.drop(columns='date')

    #Step 4 - Convert day data into numpy array
    day_test_data = np.array(day_test_data)
    day_label_data = np.array(day_label_data).reshape(-1,1)

    #Step 5 - Scale data for Neural Network
    day_test_data = x_scaler.fit_transform(day_test_data)
    day_label_data = y_scaler.fit_transform(day_label_data)

    #Step 6 - Reshape data to fit keras requirement to have a (x,y,z) shape
    day_test_data = day_test_data.reshape(1,day_test_data.shape[0],day_test_data.shape[1])
    day_label_data = day_label_data.reshape(1,day_label_data.shape[0],day_label_data.shape[1])
    
    #print(day_test_data.shape , day_label_data.shape)#debugging
    
    #Step 7 - Reshape data into Batches using slidding window   
    batch_val_size = day_test_data.shape[1] - sequence_length
    
    x_val_shape = (batch_val_size, sequence_length, num_x_signal)
    x_batch = np.zeros(shape=x_val_shape, dtype=np.float16)
        
    y_val_shape = (batch_val_size, num_y_signal)
    y_batch = np.zeros(shape=y_val_shape, dtype=np.float16)
    
    #print(x_batch.shape, y_batch.shape) # debugging
    for i in range(batch_val_size):

        # Copy the sequences of data starting at this index.
        x_batch[i] = day_test_data[0][i:i+sequence_length][:]
        y_batch[i] = day_label_data[0][i+sequence_length-1][:]

    
    return (x_batch, y_batch)

In [0]:
X_val, Y_val = batch_validation(sequence_length,num_x_signal, num_y_signal, DayTrade_clean, target_data_clean, testing_days)
print(X_val.shape, Y_val.shape)

(1405, 25, 14) (1405, 1)


In [0]:
validation_data = (X_val, Y_val)

## Create Recurrent Neural Network

In [0]:
from tensorflow.keras.layers import BatchNormalization
#from keras.constraints import max_norm

model = Sequential()
model.add(LSTM(units=200,
              return_sequences=True,
              input_shape=(None,num_x_signal,)))
model.add(LSTM(units=150, return_sequences=False))
model.add(Dense(num_y_signal,activation='relu'))
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 200)         172000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 150)               210600    
_________________________________________________________________
dense (Dense)                (None, 1)                 151       
Total params: 382,751
Trainable params: 382,751
Non-trainable params: 0
_________________________________________________________________


In [0]:
optimizer = Adam(lr=1e-3)

model.compile(loss=losses.mean_squared_error, optimizer=optimizer)

In [0]:
#es = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

model_file = "LSTM 2 layers_Relu_MinMax_modified_600epochs.h5"

mc = ModelCheckpoint(model_file, monitor="val_loss", mode="min", save_best_only=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=4, min_lr=1e-4)

In [0]:
%%time

history = model.fit_generator(generator=generator,
                    epochs=2000,
                    steps_per_epoch=50,
                    validation_data=validation_data,
                    callbacks=[ mc, reduce_lr])
                    #callbacks=[es, reduce_lr])

In [0]:
hist_df = pd.DataFrame(history.history) 
hist_csv_file = model_file.split('.')[0]+'.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [0]:
saveDrivePath = '/content/drive/My Drive/Capstone/Approach/Step2DayTrade_ver3'
h5FilePath = '/content/'+ model_file
historyFilePath = '/content/' + hist_csv_file

os.system("mv "+'"'+h5FilePath+'"' + " " + '"'+saveDrivePath+'"' )
os.system("mv "+'"'+historyFilePath+'"' + " " + '"'+saveDrivePath+'"' )

In [0]:
historyFilePath

In [0]:
plt.plot(history.history['val_loss'])
plt.title('model_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['test'], loc='upper left')
plt.show()

In [0]:
plt.plot(history.history['loss'])
plt.title('model_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [0]:
# The algorithm uses data of the previous 25 time-steps to forecast the following 10th minute into the future. Therefore,  
# the first prediction of every day can only be excecuted after 25 minutes of the begining of the day, and will refer  
# to the prediction of the minute 35.

# Since every day consists of 1440 minutes the algorithm is only able to predict the last 1405 minutes of the day.

#Create a Dataframe to hold the true predicted values for each day.
predictionsData = DayTrade.copy()
predictionsData = predictionsData.loc[:,['Weighted_Price','date']]
predictionsData = predictionsData[predictionsData['date'].isin(testing_days)]# Filter only the testing days
predictionsData = predictionsData.groupby('date').apply(lambda group: group.iloc[35:])# For each day filter the last 1405 minutes (1440 - 1405 = 35)
predictionsData.shape

In [0]:
def predict(modelFilename):

    column_name = modelFilename.split('.')[0]
    #Step 1 - Create a column to hold predictions
    predictionsData[column_name] = np.nan 
    
    #Step 2 - load model for prediction   
    loaded_model = tf.keras.models.load_model(modelFilename)
    
    for day in testing_days:
        #Step 3 - Filter Input data and Target with the selected date
        day_test_data = DayTrade_clean[DayTrade_clean['date'].values==day]

        #Step 4 - Drop date column
        day_test_data = day_test_data.drop(columns='date')

        #Step 5 - Convert day data into numpy array
        day_test_data = np.array(day_test_data)

        #Step 6 - Scale data for Neural Network
        day_test_data = x_scaler.fit_transform(day_test_data)

        #Step 7 - Reshape data to fit keras requirement to have a (x,y,z) shape
        day_test_data = day_test_data.reshape(1,day_test_data.shape[0],day_test_data.shape[1])

        #Step 8 - Reshape data into Batches using slidding window   
        batch_val_size = day_test_data.shape[1] - sequence_length

        x_val_shape = (batch_val_size, sequence_length, num_x_signal)
        x_batch = np.zeros(shape=x_val_shape, dtype=np.float16)

        for i in range(batch_val_size):

            # Copy the sequences of data starting at this index.
            x_batch[i] = day_test_data[0][i:i+sequence_length][:]

        #Step 9 - Generate the prediction for that day   
        ypred = loaded_model.predict(x_batch)
        ypred_rescaled = y_scaler.inverse_transform(ypred)

        #Step 10 - Copy the prediction values to the correspondent day in the predictionData
        predictionsData.loc[predictionsData['date']==day,column_name] = ypred_rescaled
    
    return(predictionsData)

In [0]:
modelPath = '/content/drive/My Drive/Capstone/Approach/Step2DayTrade_ver3'
modelfilePath = modelPath + '/' + model_file

In [0]:
predict(modelfilePath)

In [0]:
# Save Dataframe with prediction as csv file
prediction_file = 'predictions_' +  model_file.split('.')[0] + '.csv'
prediction_filePath = '/content/' + prediction_file
predictionsData.to_csv(prediction_file)

In [0]:
destinantionDir = '/content/drive/My Drive/Capstone/Approach/Step2DayTrade_ver3/Predictions_csv'
oscmd = "mv "+'"'+prediction_filePath+'"' + " " + '"'+destinantionDir+'"' 
oscmd

In [0]:
# Move csv files to directory in Drive
os.system(oscmd)